### Final Team Project : Multi-Agent Financial Analysis System (LangGraph)

In this segment we integrate specialized agents designed for market data retrieval, financial analysis and quantitative (Fundamental) analysis with the overarching aim of investment strategy formulation. We will develop an intelligent, automated agent system that replicates the workflow of human Investment Researcher

In [1]:
# !pip install langgraph-supervisor langchain-openai
# !pip install -U langchain-community
# ! pip install langchain_groq
# ! pip install tools
# ! pip install groq


#### 1. Library Imports and Setting up the Environment :

In [2]:
import os
import time
import torch
import json
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Langchain and LangGraph Frameworks
from langchain.tools import Tool
from langchain_groq import ChatGroq
from langchain.tools import StructuredTool
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain.document_loaders.web_base import WebBaseLoader
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool


In [3]:
GROQ_API_Key = "<YOUR API KEY>"
# Set up GROQ API Key as Environmental variable
os.environ["GROQ_API_KEY"] = GROQ_API_Key

#### 2. Defining Specialized Agent Functions :

##### Market and Business News Analysis :   

In [4]:
def Format_results(docs, query):
  title_content_list = []
  # Iterate through the loaded web pages and format their respective titles and content
  for doc in docs:
    title = doc.metadata.get('title','No title available')
    page_content = doc.page_content.strip() if query in doc.page_content else "" # Removing unnecessary training/leading spaces
    title_content = f"{title}:{page_content}\n"
    title_content_list.append(title_content)
  # Join formatted content and titles into a single string
  return "\n".join(title_content_list)

In [5]:
# Retrieving Market news
# Function to retrieve Stock symbol and stock related news from Yahoo Finance
def Financial_News(ticker, n_search_results=2):
  """ Simulating fetching financial news for a given stock """
  # Retrieve the news from Yahoo Finance
  links = []
  try:
    # Create a yfinance.Ticker object for the specified ticker
    company = yf.Ticker(ticker)
    # Retrieving news articles of type "STORY" and storing their respective links
    links = [n["link"] for n in company.news if n["type"] == "STORY"]
    print(f"Links are retrieved and stored successfully")
  except:
    print(f"No news found from Yahoo Finance")

  # Create a WebBaseLoader to load web pages using the collected links
  loader = WebBaseLoader(links)
  docs = loader.load()
  # Formatting the results by combining titles and page content
  data = Format_results(docs, ticker)
  print(f"Completed retrieving news for {ticker}")
  return data


##### Market data and Financial Analysis :

In [6]:
def Financial_Statements(ticker):
  """ Here we simulate fetching stock market data for a given stock symbol. """
  # Creating a Ticker object for the specified stock ticker
  company = yf.Ticker(ticker)

  # Modules to fetch company's balance sheet, cash flow and income statement data
  balance_sheet_statement = company.balance_sheet
  cash_flow_statement = company.cash_flow
  income_statement = company.income_stmt

  # Set up the file name with csv pre-fix
  csv_file_prefix = f"{ticker}_financial_"

  # Retrieve the Stock price data
  stock_data = yf.download(ticker, period='1y', interval='1d')

  # Saving stock price data to the CSV file
  data_csv_filename = csv_file_prefix + "stock_data.csv"
  stock_data.to_csv(data_csv_filename)

  # Save financial statements to the CSV file
  balance_sheet_statement_csv_filename = csv_file_prefix + "balance_sheet_statement.csv"
  cash_flow_statement_csv_filename = csv_file_prefix + "cash_flow_statement.csv"
  income_statement_csv_filename = csv_file_prefix + "income_statement.csv"

  # Save the data from balance sheet, cash flow, and income statement
  balance_sheet_statement.to_csv(balance_sheet_statement_csv_filename)
  cash_flow_statement.to_csv(cash_flow_statement_csv_filename)
  income_statement.to_csv(income_statement_csv_filename)

  print(f"Financial Statements and Stock Price data are saved in respective CSV files")
  return data_csv_filename, balance_sheet_statement_csv_filename, cash_flow_statement_csv_filename, income_statement_csv_filename


##### Quantitative Analysis Metrics :

In [7]:
def Fundamental_Analysis_Statements(ticker):
  """
  Compute P/E ratio, ROE (Return on Investment) and Revenue growth
  Function to compute key fundamental ratios for ticker using yfinance.
  Returns a dict with:
  - 'info': ticker.info (metadata)
  - 'financials': 'balance_sheet','cashflow','Income Statement'(DataFrames)
  - 'ratios': Computed key fundamental ratios
  - 'notes': Any warning/messages
  """
  company = yf.Ticker(ticker)
  result = {'ticker':ticker, 'info':{}, 'financials':None, 'balance_sheet':None, 'cashflow':None,'ratios':{},'notes':[]}

  try:
    info = company.info or {}
    result['info'] = info
  except Exception as e:
    result['notes'].append(f"Unable to retrieve ticker.info: {e}")

  # Financial Statements (DataFrames)
  try:
    income_statement = company.income_stmt
    balance_sheet = company.balance_sheet
    cash_flow = company.cash_flow

    result['financials'] = income_statement
    result['balance_sheet'] = balance_sheet
    result['cash_flow'] = cash_flow

  except Exception as e:
    result['notes'].append(f"Unknown error: Unable to fetch statements: {e}")

  try:
    # Fundamental Ratios
    r = {}
    r['marketCap'] = info.get('marketCap')
    r['trailingPE'] = info.get('trailingPE')
    r['currentPrice'] = info.get('currentPrice')
    r['trailingEps'] = info.get('trailingEps')
    # Calculating and recording PE Ratio as follows :-
    r['pe_ratio'] = r['currentPrice']/r['trailingEps']

    # Calculating ROE from the Balance Sheet and Income Statement
    # ROE = Net Income / Total Stockholder Equity
    net_income = company.income_statement.loc['Net Income', income_statement.columns[0]]
    shareholders_equity = company.balance_sheet.loc['Total Stockholder Equity', balance_sheet.columns[0]]

    if pd.notna(net_income) and pd.notna(shareholders_equity) and shareholders_equity != 0:
        ROE = float(net_income/shareholders_equity)*100
        r['ROE'] = float(net_income/shareholders_equity)*100
    else:
        r['ROE'] = 0

    # Calculating Operating Margin
    revenue = company.income_stmt.loc['Total Revenue']
    operating_expenses = company.income_stmt.loc['Total Operating Expenses']
    if pd.notna(revenue) and pd.notna(operating_expenses) and revenue != 0:
      operating_margin = float((revenue - operating_expenses) / revenue) * 100
      r['operating_margin'] = float((revenue - operating_expenses) / revenue) * 100
    else:
      r['operating_margin'] = 0

    result['ratios'] = r

    # Data Visualization :-
    eps_history = company.income_stmt.loc['Net Income']
    # Plot historical EPS
    eps_history.plot(kind='bar', figsize=(6,5))
    plt.title('Stock Quarterly EPS over time')
    plt.xlabel('Quarter')
    plt.ylabel('Earnings per Share($)')
    plt.tight_layout()
    plt.show()

    # Plotting operating margin
    operating_margin.plot(kind='bar', figsize=(6,5))
    plt.title(f'Quarterly Operating Margin for {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Operating Margin (%)')
    plt.tight_layout()
    plt.show()

    # Plotting ROE
    ROE.plot(kind='bar', figsize=(6,5))
    plt.title(f'Quarterly Return on Equity (ROE) for {ticker}')
    plt.xlabel('Date')
    plt.ylabel('ROE (%)')
    plt.tight_layout()
    plt.show()


  except Exception as e:
    result['notes'].append(f"Data Unavailable. Please check yfinance Documentation for data points availability.")

  return result


In [8]:
def Fundamental_Analysis(ticker):
  """
  JSON serializable strings for Analysis statements so that it is simple for the LLMs to consume.
  """
  ticker = ticker.strip().upper()
  try:
    result = Fundamental_Analysis_Statements(ticker)
    output = {
        'ticker': result['ticker'],
        'summary':{
            'marketCap': result['ratios'].get('marketCap'),
            'trailingPE': result['ratios'].get('trailingPE'),
            'currentPrice': result['ratios'].get('currentPrice'),
            'trailingEps': result['ratios'].get('trailingEps'),
            'Operating Margin': result['ratios'].get('operating_margin'),
            'PE Ratio': result['ratios'].get('pe_ratio'),
            'ROE Ratio': result['ratios'].get('ROE')
        },
        'notes': result.get('notes', [])
    }
    return output
  except Exception as e:
    return json.dumps({'error': str(e)})


#### 3. Configuring and Deploying Agents :

In [9]:
model = ChatGroq(
    model = "llama-3.1-8b-instant",
    api_key = os.environ["GROQ_API_KEY"],
    max_tokens = 512, # Maximum number of that can be generated by the model over a single run
    temperature = 0.1, # To ensure precise response. It indicates randomness of token generation where a lower value results in more deterministic token generation.
    tool_choice = "none",
    tools = []
)

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:253: UserWarning: WARNING! tool_choice is not default parameter.
                    tool_choice was transferred to model_kwargs.
                    Please confirm that tool_choice is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:253: UserWarning: WARNING! tools is not default parameter.
                    tools was transferred to model_kwargs.
                    Please confirm that tools is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


Here we set additional parameters namely

tool_choice = "none", tools = [] inside our model definition.

This will restrict the supervisor application to registered agents only.

The option otherwise is to registed in BraveSearch and generate an API Key for enabling additional web-browsing for financial data. It can be used in conjunction with models such as defined above : Groq or OpenAI.

N:B - In OpenAI style models we can apply the above restriction setting the flag enable_tool_use = False

Alternatively, we can allow web-browsing for financial data as follows -


from langchain_community.tools.brave_search import BraveSearch

######## Set your Brave API key

os.environ["BRAVE_API_KEY"] = "<YOUR_BRAVE_API_KEY>"

######### Create the tool

brave_search_tool = BraveSearch(api_key=os.environ["BRAVE_API_KEY"], max_results=5)


##### Create Agents

Supervisor Agent Implementation (With no Reasoning Chains) -

In [10]:
# # News Data Agent
# news_analysis_expert = create_react_agent(
#     model = model,
#     tools = [Financial_News],
#     name = "news_analysis_expert",
#     prompt = "You analyze financial news for given stock symbols. Return structured insights only."
# )

# # Finance Data Analysis Agent
# finance_analysis_expert = create_react_agent(
#     model = model,
#     tools = [Financial_Statements],
#     name = "finance_analysis_expert",
#     prompt = "You are an expert in stock market data. Fetch and analyze stock data such as balance sheet and income statement when requested."
#              "You are a financial statements expert. Always return structured data from Financial_Statements. Do not speculate beyond the data. "
# )

# # Quantitative Analysis Agent
# quant_expert = create_react_agent(
#     model = model,
#     tools = [Fundamental_Analysis],
#     name = "quant_expert",
#     prompt = "You are a quantitative analyst. Use the provided tools to calculate and report key metrics. "
#              "like P/E ratio, ROE, and operating margins."
#              "Generate visuals for metrics defined inside the tools."
#              "Return actual values from the tool output, not general statements."
#              "Avoid assumptions or subjective recommendations."
# )



In [11]:
# #### SUPERVISOR Agent ####
# market_research_supervisor = create_supervisor(
#     agents = [news_analysis_expert, finance_analysis_expert, quant_expert],
#     model = model,
#     prompt = (
#         "You are a financial market supervisor managing three expert agents: news, finance, and quantitative (Fundamental) analysis."
#         "Use news_analysis_expert (for news). Use finance_analysis_expert (for financial statements)"
#         "and use quant_expert (for key ratios)."
#         "Additionally, as a financial researcher use the agents' numeric data (like P/E, ROE, revenue growth, margins, etc.) to form your insights."
#         "Utilize the data analysis done on behalf of the above-mentioned agents to create a detailed investment thesis to address the user's request."
#         "Always reference actual figures from their outputs. "
#         "Please back your assertions with substantial data and analysis. Do NOT generate unsupported claims or general statements like 'strong balance sheet'."
#         "Provide factual analysis grounded in the agents' data."
#         "You refrain from providing direct 'Buy' or 'Sell' recommendations to comply with legal regulations."

#     )
# )

# # Compile into an executable workflow
# application = market_research_supervisor.compile()

##### Supervisor Implementation (with Reasoning chains) :

In [12]:
# # News Data Agent
# news_analysis_expert = create_react_agent(
#     model = model,
#     tools = [Financial_News],
#     name = "news_analysis_expert",
#     prompt = "You analyze financial news for given stock symbols. Return structured insights only."
# )

# # Finance Data Analysis Agent
# finance_analysis_expert = create_react_agent(
#     model = model,
#     tools = [Financial_Statements],
#     name = "finance_analysis_expert",
#     prompt = "You are an expert in stock market data."
#              "You are a financial statements expert. Always return structured data from Financial_Statements for the requested ticker. Avoid opinions.Do not speculate beyond the data. "
# )

# # Quantitative Analysis Agent
# quant_expert = create_react_agent(
#     model = model,
#     tools = [Fundamental_Analysis],
#     name = "quant_expert",
#     prompt = "You are a quantitative analyst. Use the provided tools to calculate and report key metrics"
#              "like P/E ratio, ROE, and operating margins."
#              "Generate visuals for metrics defined inside the tools."
#              "Return actual values from the tool output. Return structured values only; do not speculate."
#              "Avoid assumptions or subjective recommendations."
# )



In [13]:
# #### SUPERVISOR Agent ####
# market_research_supervisor = create_supervisor(
#     agents = [news_analysis_expert, finance_analysis_expert, quant_expert],
#     model = model,
#     prompt = (
#         "You are a financial market supervisor managing three expert agents: news_analysis_expert, finance_analysis_expert, and quant_expert(Fundamental analysis)."
#         "Your job is to analyze user query and decide which agent(s) to call or invoke in order"
#         "Use the following reasoning chain to gather relevant data."
#         "1. Determine the type of information needed (news, financial statements, or ratios).\n"
#         "2. Call only the required agents for the given user query.\n"
#         "3. Collect structured outputs from the agents.\n"
#         "4. Analyze the collected data to produce a final structured report.\n"
#         "Utilize the data analysis done on behalf of the above-mentioned agents to create a detailed investment thesis to address the user's request."
#         "Always reference actual data from the agent outputs."
#         "Please back your assertions with substantial data and analysis. Do NOT generate unsupported claims or general statements like 'strong balance sheet'."
#         "Provide factual analysis grounded in the agents' data."
#         "You refrain from providing direct 'Buy' or 'Sell' recommendations to comply with legal regulations."
#         "Example reasoning: 'User asked about NVDA financial ratios -> call quant_expert and finance_analysis_expert only.'"

#     )
# )

# # Compile into an executable workflow
# application = market_research_supervisor.compile()

Supervisor with Evaluator-Optimizer Implementation :-

In [14]:
# News Data Agent
news_analysis_expert = create_react_agent(
    model = model,
    tools = [Financial_News],
    name = "news_analysis_expert",
    prompt = "You analyze financial news for given stock symbols. Return structured insights only."
)

# Finance Data Analysis Agent
finance_analysis_expert = create_react_agent(
    model = model,
    tools = [Financial_Statements],
    name = "finance_analysis_expert",
    prompt = "You are an expert in stock market data."
             "You are a financial statements expert. Always return structured data from Financial_Statements for the requested ticker. Avoid opinions.Do not speculate beyond the data. "
)

# Quantitative Analysis Agent
quant_expert = create_react_agent(
    model = model,
    tools = [Fundamental_Analysis],
    name = "quant_expert",
    prompt = "You are a quantitative analyst. Use the provided tools to calculate and report key metrics"
             "like P/E ratio, ROE, and operating margins."
             "Generate visuals for metrics defined inside the tools."
             "Return actual values from the tool output. Return structured values only; do not speculate."
             "Avoid assumptions or subjective recommendations."
)



In [15]:
#### SUPERVISOR Agent ####
market_research_supervisor = create_supervisor(
    agents = [news_analysis_expert, finance_analysis_expert, quant_expert],
    model = model,
    prompt = (
        "You are a financial market supervisor managing three expert agents: news_analysis_expert, finance_analysis_expert, and quant_expert(Fundamental analysis)."
        "Your job is to analyze user query and decide which agent(s) to call or invoke in order"
        "Use the following reasoning chain to gather relevant data."
        "1. Determine the type of information needed (news, financial statements, or ratios).\n"
        "2. Call only the required agents for the given user query.\n"
        "3. Collect structured outputs from the agents.\n"
        "4. Analyze the collected data to produce a final structured report.\n"
        "Utilize the data analysis done on behalf of the above-mentioned agents to create a detailed investment thesis to address the user's request."
        "Always reference actual data from the agent outputs."
        "Please back your assertions with substantial data and analysis. Do NOT generate unsupported claims or general statements like 'strong balance sheet'."
        "Provide factual analysis grounded in the agents' data."
        "You refrain from providing direct 'Buy' or 'Sell' recommendations to comply with legal regulations."
        "Example reasoning: 'User asked about NVDA financial ratios -> call quant_expert and finance_analysis_expert only.'"

    )
)

# Compile into an executable workflow
application = market_research_supervisor.compile()

###### Agent Wrapping as a Tool :

In [16]:
def run_news_analysis(query: str) -> str:
    """Run the News Analysis Agent"""
    result = news_analysis_expert.invoke({"input": query})
    return result["output"]

def run_finance_analysis(query: str) -> str:
    """Run the Financial Statements Agent"""
    result = finance_analysis_expert.invoke({"input": query})
    return result["output"]

def run_quant_analysis(query: str) -> str:
    """Run the Fundamental/Quantitative Agent"""
    result = quant_expert.invoke({"input": query})
    return result["output"]

# Wrap as structured tools
news_tool = StructuredTool.from_function(run_news_analysis)
finance_tool = StructuredTool.from_function(run_finance_analysis)
quantitative_tool = StructuredTool.from_function(run_quant_analysis)

In [17]:
# Creating Evaluator Agent
evaluator_agent = create_react_agent(
    model = model,
    tools = [], # works only with supervisor output
    name = "evaluator_agent",
    prompt=(
        "Yor are an evaluator for evaluating financial reports.\n "
        "For a given supervisor-generated report, check the following :\n"
        "- Completeness (all requested metrics included)\n"
        "- Consistency (no conflicting numbers)\n"
        "- Clarity and structure\n"
        "Return structured feedback emphasizing required improvements."
    )
)

In [18]:
optimizer_agent = create_react_agent(
    model = model,
    tools = [news_tool, finance_tool, quantitative_tool], # Can fetch data when required
    name = "optimizer_agent",
    prompt = (
        "You are an optimizer of financial reports. \n"
        "Using evaluator feedback, refine the report by: \n"
        "- Adding missing metrics\n"
        "- Improving clarity and organization\n"
        "- Incorporating additional agent outputs if required\n"
        "Ensure final report is fully grounded on actual agent data."
    )
)

##### Post-Processing Agent Outputs and Summarization (Optional) Step

In [19]:
# # Optionally adding a Post-Processor of Multi-Agent outputs before feed-back to the Supervisor
# def agent_output_summarization(outputs):
#   """ Combine results acquired from all agents into a single input for the supervisor."""
#   summary = []
#   for name, data in outputs.items():
#     summary.append(f"--{name} ---\n{json.dumps(data, indent=1)}")
#   return "\n".join(summary)

#### 4. Running the Application :

In [20]:
# Supervisor Implementation with no Reasoning chaining with prompts and No Evaluator-Optimizer implementation
# system_instruction = {
#     "role": "system",
#     "content": (
#         "You are restricted to using only the tools defined by the user. "
#         "You do NOT have any web search or internet browsing capability. "
#         "If you need financial data, reason from known fundamentals instead."
#     ),
# }
# stock_query = {
#     "messages":[
#         system_instruction,
#         {"role":"system","content":"You may only use tools defined by the user."},
#         {"role":"user", "content":"Analyze APPL's financials and provide an investment summary."}
#     ]
# }

# # Execute query
# result = application.invoke(stock_query)
# print(result['messages'][-1].content)

In [21]:
# Supervisor Implementation - With optional Summarative function
# agent_outputs = {
#     "news": news_analysis_expert.invoke({"messages":[{"role":"user","content":"AAPL"}]})["messages"][-1].content,
#     "finance": finance_analysis_expert.invoke({"messages":[{"role":"user","content":"AAPL"}]})["messages"][-1].content,
#     "quant": quant_expert.invoke({"messages":[{"role":"user","content":"AAPL"}]})["messages"][-1].content,
# }

# # Now these are plain strings, not message objects
# combined_summary = agent_output_summarization(agent_outputs)

# result = application.invoke({
#     "messages": [
#         {"role":"user", "content": f"Analyze AAPL using these results:\n{combined_summary}"}
#     ]
# })

# print(result["messages"][-1].content)


In [22]:
# Supervisor implementation with Reasoning Chain - No Evaluator-Optimizer Implementation
# # Helper Function for executing dynamic reasoning
# def stock_analysis(ticker):
#   stock_query = {
#       "messages":[
#           {"role":"user","content":f"Analyze {ticker}'s news, financial and quantitative metrics."}
#       ]
#   }
#   result = market_research_supervisor.compile().invoke(stock_query)
#   return result['messages'][-1].content

In [23]:
# nvda_report = stock_analysis("NVDA")
# print(nvda_report)


In [24]:
def autonomous_stock_analysis(ticker, max_iterations=3):
  stock_query = {
      "messages":[
          {"role":"user","content":f"Provide a detailed financial analysis for {ticker}, including news, financial statements and key quantitative metrics."}
      ]
  }
  supervisor_output = application.invoke(stock_query)['messages'][-1].content
  final_report = supervisor_output
  iteration = 0

  while iteration < max_iterations:
    iteration = iteration + 1

    evaluator_output = evaluator_agent.invoke({
            "messages": [
                {"role": "user", "content": final_report}
            ]
        })['messages'][-1].content

    if "No issues" in evaluator_output or "complete" in evaluator_output.lower():
            break
    final_report = optimizer_agent.invoke({
            "messages": [
                {"role": "user", "content": f"Refine the following report based on feedback:\nReport:\n{final_report}\nEvaluator Feedback:\n{evaluator_output}"}
            ]
        })['messages'][-1].content

  return final_report

In [25]:
nvda_report = autonomous_stock_analysis("NVDA")
print(nvda_report)


To provide a detailed financial analysis for NVDA, I will follow the reasoning chain outlined earlier.

1. Determine the type of information needed: The user has requested a comprehensive analysis, which includes news, financial statements, and key quantitative metrics. This requires calling all three agents: news_analysis_expert, finance_analysis_expert, and quant_expert.

2. Call the required agents:
- news_analysis_expert: To gather recent news and updates about NVDA.
- finance_analysis_expert: To analyze NVDA's financial statements and provide insights on their financial health.
- quant_expert: To calculate key quantitative metrics, such as financial ratios and performance indicators.

3. Collect structured outputs from the agents:

- news_analysis_expert output:
NVDA has recently announced a partnership with a leading technology company to develop advanced AI chips. This partnership is expected to boost NVDA's revenue and market share in the AI chip segment. Additionally, NVDA has